# Importando bibliotecas

In [36]:
# biblotecas para manipulação de dados
import pandas as pd
import datatable as dt
import numpy as np

#Bibliotecas auxiliares
import os
from functools import reduce
import re
import pickle
import gc
import tqdm

#Biblioteca propria
import sys
sys.path.append("../src/")
from eda.eda import describe
from io_pyarrow.io_pyarrow import pyarrow_read_csv,write_table_from_pandas, read_table_to_pandas #leitura e escrita de arquivos csv grandes

%reload_ext watermark
%watermark --iversions

tqdm     : 4.60.0
datatable: 0.11.1
pandas   : 1.2.2
re       : 2.2.1
sys      : 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
numpy    : 1.20.1



___

# Importando arquivo

In [2]:
#Definindo path dos arquivos
path_data = "../data/raw/"

## Dados da FAPESP
<b> Descrição </b>: Pasta de arquivos, com dados anonimizados de pacientes que fizeram teste para COVID-19 a partir de 1/11/2019 e respectivos resultados de exames laboratoriais, contendo dentre outros o identificador anonimizado do paciente  <br>
fonte: https://repositoriodatasharingfapesp.uspdigital.usp.br/

### Grupo Fleury

In [3]:
%%time
fleury_exames = pyarrow_read_csv(path_data + "FAPESP/GrupoFleury_Exames_3.csv",sep="|")
fleury_pacientes = pyarrow_read_csv(path_data + "FAPESP/GrupoFleury_Pacientes_3.csv",sep="|")

Wall time: 1min


___

# Analisando a bases de dados

### Ajustando dados do Fleury

In [4]:
fleury = fleury_exames.merge(fleury_pacientes,
                             on = "ID_PACIENTE",
                             how = 'left')
del(fleury_exames)
del(fleury_pacientes)
describe(fleury)

Quantidade de linhas: 19274381


,variable,type,na,na_pct,unique,min,quat25,median,mean,quat75,max,std,skewness,kurtosis,media_desvio
0,ID_PACIENTE,object,0,0.0%,470969,-,-,-,-,-,-,-,-,-,-
1,ID_ATENDIMENTO,object,0,0.0%,952463,-,-,-,-,-,-,-,-,-,-
2,DT_COLETA,object,0,0.0%,411,-,-,-,-,-,-,-,-,-,-
3,DE_ORIGEM,object,0,0.0%,1,-,-,-,-,-,-,-,-,-,-
4,DE_EXAME,object,0,0.0%,830,-,-,-,-,-,-,-,-,-,-
5,DE_ANALITO,object,0,0.0%,1158,-,-,-,-,-,-,-,-,-,-
6,DE_RESULTADO,object,0,0.0%,32613,-,-,-,-,-,-,-,-,-,-
7,CD_UNIDADE,object,0,0.0%,57,-,-,-,-,-,-,-,-,-,-
8,DE_VALOR_REFERENCIA,object,0,0.0%,1102,-,-,-,-,-,-,-,-,-,-
9,IC_SEXO,object,94,0.0%,3,-,-,-,-,-,-,-,-,-,-


In [5]:
#salvando o arquivo, pois é um dataframe muito grande
write_table_from_pandas(fleury,"../data/interim/raw_fleury.parquet")

In [3]:
%%time
fleury = read_table_to_pandas("../data/interim/raw_fleury.parquet")

Wall time: 17.7 s


In [4]:
fleury_ = fleury.copy()

#### DT_COLETA

In [5]:
#DT_COLETA é um dado no formato de data que deve ser convertido
fleury_.DT_COLETA = pd.to_datetime(fleury.DT_COLETA,errors='coerce')

In [6]:
fleury_.DT_COLETA.isna().sum()

0

#### DE_ORIGEM

In [7]:
#Possui apenas um valor, portanto não há necessidade de manter essa informação
fleury_ = fleury_.drop(columns = "DE_ORIGEM")

#### CD_UNIDADE

In [8]:
#Existem alguns casos vazios para CD_Unidade
fleury_.CD_UNIDADE.value_counts()

                       4913738
mg/dL                  2990870
%                      2250992
/mm3                   2012392
g/dL                    902600
U/L                     850142
UA/mL                   745771
fL                      553192
ng/dL                   502019
ng/mL                   353048
pg                      281537
milhões/mm3             281537
g/L                     267367
mEq/L                   265482
microg/dL               255886
microg/L                246419
mUI/L                   213458
segundos                205510
UI/L                    157782
U/mL                    145587
mU/L                    139788
ng/L                    113787
UI/mL                    87461
mmol/L                   75567
pg/mL                    66243
pmol/L                   61493
mm                       58754
micromol/L               55623
kU/L                     53591
nmol/L                   44004
por campo                36506
microgramas/mL           30811
mg/L    

In [9]:
#Existem algumas unidades vazias, portanto vamos aplicar NaN a elas e transformar em categoria
fleury_.CD_UNIDADE = fleury_.CD_UNIDADE.replace({"":np.nan})
fleury_.CD_UNIDADE = pd.Categorical(fleury_.CD_UNIDADE,categories=fleury_.CD_UNIDADE.value_counts().index)

#### DE_VALOR_REFERENCIA

In [10]:
#Existem alguns valores vazios em valor de referencia
fleury_.DE_VALOR_REFERENCIA.value_counts()

                     4699971
Não reagente         1173352
inferior a 0,8        717949
150.000 a 450.000     280420
1.700 a 7.000         278126
                      ...   
155 a 275                  1
29 a 161                   1
5 a 34                     1
100 a 810                  1
Até 0,8                    1
Name: DE_VALOR_REFERENCIA, Length: 1102, dtype: int64

In [11]:
# Vamos pegar esses valores em branco e transformá-los
fleury_.DE_VALOR_REFERENCIA = fleury_.DE_VALOR_REFERENCIA.replace({"":np.nan})

#### IC_SEXO

In [12]:
#Existem alguns casos vazios, que foram provenientes da tabela de pacientes, mas são poucos os casos
fleury_.IC_SEXO.value_counts(dropna=False)

F      12004191
M       7270096
NaN          94
Name: IC_SEXO, dtype: int64

In [13]:
# Convertendo para dados categoricos
fleury_.IC_SEXO = pd.Categorical(fleury_.IC_SEXO,categories=["F","M"])

#### AA_NASCIMENTO

In [14]:
#Há valores que não podem ser convertidos para numero. Conforme o dicionario de dados, AAAA podem aparecer para anonimizar as informações
pd.to_numeric(fleury_.AA_NASCIMENTO.value_counts().index,errors='coerce',downcast='integer')
fleury_.AA_NASCIMENTO = pd.to_numeric(fleury_.AA_NASCIMENTO,errors='coerce')

#### CD_PAIS

In [15]:
#Só há um país, portanto iremos remover essa info
fleury_.CD_PAIS.value_counts()

BR    19274287
Name: CD_PAIS, dtype: int64

In [16]:
fleury_ = fleury_.drop(columns = "CD_PAIS")

#### CD_UF

In [17]:
#Existe uma UF UU, que deve ser removida
fleury_.CD_UF.value_counts()

SP    11608391
RJ     4613909
RS     1283827
PE      782501
PR      624400
BA      252657
UU       43268
DF       30411
MG       11544
MA        7414
SC        5275
RN        4052
GO        2958
AM        1066
PA         915
ES         694
AC         459
CE         252
MT         119
PB          95
MS          80
Name: CD_UF, dtype: int64

In [18]:
fleury_.CD_UF = pd.Categorical(fleury_.CD_UF.replace({"UU":np.nan}), categories=fleury_.CD_UF.replace({"UU":np.nan}).value_counts().index)

#### CD_MUNICIPIO

In [19]:
#Existem municipios MMMM que ocorrem quando houver necessidade de  anonimização ou residente no exterior, conforme informações do dicionario de dados
fleury_.CD_MUNICIPIO.value_counts()

SAO PAULO         8523885
RIO DE JANEIRO    3586654
PORTO ALEGRE      1021221
MMMM               701631
RECIFE             644701
                   ...   
RIB. PRETO             20
JOAO PESSOA            18
FORTALEZA              16
GOIANA                 16
RIB PRETO              10
Name: CD_MUNICIPIO, Length: 134, dtype: int64

In [20]:
fleury_.CD_MUNICIPIO = fleury_.CD_MUNICIPIO.replace({"MMMM":np.nan})

#### CD_CEPREDUZIDO

In [21]:
#70% dos CEP são inexistentes, portanto iremos remover essa coluna
fleury_.CD_CEPREDUZIDO.value_counts(normalize=True)*100

CCCC     70.021324
60299     1.465937
24220     0.772459
24230     0.558044
51020     0.493647
           ...    
07074     0.000052
14060     0.000052
81830     0.000052
42802     0.000052
13309     0.000052
Name: CD_CEPREDUZIDO, Length: 1150, dtype: float64

In [22]:
fleury_ = fleury_.drop(columns = "CD_CEPREDUZIDO")

#### DE_RESULTADO

De acordo com o dicionário de dados:

<i>"Se DE_ANALITO exige valor numérico, NNNN se inteiro ou NNNN,NNN se casas decimais
Se DE_ANALITO exige qualitativo, String com domínio restrito 
Se DE_ANALITO por observação microscópica, String conteúdo livre "</i>

Portanto, vamos fazer essa divisão

In [23]:
fleury_.DE_RESULTADO.value_counts()

NÃO REAGENTE                                                                                                                                                                                                                 816213
0,1                                                                                                                                                                                                                          447554
negativa                                                                                                                                                                                                                     372826
não reagente                                                                                                                                                                                                                 330950
não foram observados caracteres tóxico-degenerativos nos neutrófilos, não foram observad

In [24]:
def func_filtro_numerico_DE_RESULTADO(x):
    if((re.search("[0-9]+",x.lower()) == None) & (re.search("[0-9]+,[0-9]+",x.lower()) == None)):
        return False
    elif(re.search("[0-9]+,[0-9]+",x.lower()) != None):
        if(re.search("[0-9]+,[0-9]+",x.lower()).span()[1] - re.search("[0-9]+,[0-9]+",x.lower()).span()[0] == len(x)):
            return True
        else:
            return False
    else:
        if(re.search("[0-9]+",x.lower()).span()[1] - re.search("[0-9]+",x.lower()).span()[0] == len(x)):
            return True
        else:
            return False

In [25]:
filtro_numerico_DE_RESULTADO = fleury_.DE_RESULTADO.apply(func_filtro_numerico_DE_RESULTADO)

In [26]:
fleury_["DE_RESULTADO_NUMERICO"] = pd.to_numeric(fleury_.DE_RESULTADO[filtro_numerico_DE_RESULTADO].str.replace(",","."),errors="coerce")
fleury_["DE_RESULTADO_NAO_NUMERICO"] = fleury_.DE_RESULTADO[~filtro_numerico_DE_RESULTADO]

In [27]:
describe(fleury_)

Quantidade de linhas: 19274381


,variable,type,na,na_pct,unique,min,quat25,median,mean,quat75,max,std,skewness,kurtosis,media_desvio
0,ID_PACIENTE,object,0,0.0%,470969,-,-,-,-,-,-,-,-,-,-
1,ID_ATENDIMENTO,object,0,0.0%,952463,-,-,-,-,-,-,-,-,-,-
2,DT_COLETA,datetime64[ns],0,0.0%,411,-,-,-,-,-,-,-,-,-,-
3,DE_EXAME,object,0,0.0%,830,-,-,-,-,-,-,-,-,-,-
4,DE_ANALITO,object,0,0.0%,1158,-,-,-,-,-,-,-,-,-,-
5,DE_RESULTADO,object,0,0.0%,32613,-,-,-,-,-,-,-,-,-,-
6,CD_UNIDADE,category,4913738,25.5%,57,-,-,-,-,-,-,-,-,-,-
7,DE_VALOR_REFERENCIA,object,4699971,24.4%,1102,-,-,-,-,-,-,-,-,-,-
8,IC_SEXO,category,94,0.0%,3,-,-,-,-,-,-,-,-,-,-
9,AA_NASCIMENTO,float64,99511,0.5%,91,1931.0,1964.0,1977.0,1975.13,1986.0,2020.0,16.69,-0.18,-0.21,118.33


In [28]:
del(fleury)

In [29]:
#Como o dataframe é muito grande, vamos separar as demais colunas nao analisadas
write_table_from_pandas(fleury_,"../data/interim/fleury.parquet")

In [30]:
demais_fleury = fleury_[["DE_ANALITO","DE_EXAME"]]

In [31]:
del(fleury_)

#### DE_ANALITO

In [5]:
demais_fleury = read_table_to_pandas("../data/interim/fleury.parquet")[["DE_ANALITO","DE_EXAME"]]

In [ ]:
# O campo analito possui mais de uma informação, por isso, vamos separa num dataframe a parte
pickle.dump(obj=np.unique([ana for elem in demais_fleury.DE_ANALITO.apply(lambda x: x.split(", ")).tolist() for ana in elem]),
            file=open("../data/interim/fleury_exames_analito/partes/de_analito.pickle","wb"))

In [15]:
# A quantidade de linhas é muito grande, portanto iremos dividir essa quantidade em arquivos menores
for i,df in enumerate(np.array_split(demais_fleury,19)):
    write_table_from_pandas(df,"../data/interim/fleury_exames_analito/partes/parte"+str(i)+".parquet")

In [ ]:
del(demais_fleury)

In [11]:
analito = pickle.load(open("../data/interim/fleury_exames_analito/partes/de_analito.pickle","rb"))

In [75]:
#Criando um DataFrame no estilo OneHotEncoding para analito
for i in tqdm.tqdm(range(19)):
    if("bool_parte"+str(i)+".parquet" in os.listdir("../data/interim/fleury_exames_analito/analito/onehot_encoding")):
        continue
    else:
        demais_parte = read_table_to_pandas("../data/interim/fleury_exames_analito/partes/parte"+str(i)+".parquet")[["DE_ANALITO"]]
        write_table_from_pandas(pd.DataFrame({chave: demais_parte.DE_ANALITO.str.contains(chave).to_numpy() for chave in analito}),
                                "../data/interim/fleury_exames_analito/analito/onehot_encoding/bool_parte"+str(i)+".parquet")

  0%|                                                                                           | 0/19 [00:00<?, ?it/s]

0
1


  5%|████▎                                                                              | 1/19 [00:02<00:49,  2.74s/it]


KeyboardInterrupt: 

#### DE_EXAME

In [ ]:
demais_fleury = read_table_to_pandas("../data/interim/fleury.parquet")[["DE_EXAME"]]

In [57]:
np.concatenate((exames,read_table_to_pandas("../data/interim/fleury_exames_analito/partes/parte"+str(i)+".parquet")["DE_EXAME"].to_numpy()))

array([None, 'NOVO CORONAVÍRUS 2019 (SARS-CoV-2), DETECÇÃO POR PCR',
       'NOVO CORONAVÍRUS 2019 (SARS-CoV-2), DETECÇÃO POR PCR', ...,
       'HTLV1/2, ANTICORPOS, soro',
       'SIFILIS, ANTICORPOS TOTAIS ESPECIFICOS ANTI-T.PALLIDUM, soro',
       'SIFILIS, ANTICORPOS TOTAIS ESPECIFICOS ANTI-T.PALLIDUM, soro'],
      dtype=object)

In [62]:
exames = np.array([],dtype=np.object_)
for i in tqdm.tqdm(range(19)):
    exame_parte = read_table_to_pandas("../data/interim/fleury_exames_analito/partes/parte"+str(i)+".parquet")[["DE_EXAME"]]
    exames = np.unique(np.concatenate((exames,
                                      np.unique([ex for elem in exame_parte.DE_EXAME.apply(lambda x: x.split(", ")).tolist() for ex in elem]))))

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [01:40<00:00,  5.30s/it]


In [63]:
# O campo exame possui mais de uma informação, por isso, vamos separa num dataframe a parte
pickle.dump(obj=exames,
            file=open("../data/interim/fleury_exames_analito/partes/de_exame.pickle","wb"))

In [64]:
exames = pickle.load(open("../data/interim/fleury_exames_analito/partes/de_exame.pickle","rb"))

In [ ]:
#Criando um DataFrame no estilo OneHotEncoding para exames
for i in tqdm.tqdm(range(19)):
    if("bool_parte"+str(i)+".parquet" in os.listdir("../data/interim/fleury_exames_analito/exames/onehot_encoding")):
        continue
    else:
        demais_parte = read_table_to_pandas("../data/interim/fleury_exames_analito/partes/parte"+str(i)+".parquet")[["DE_EXAME"]]
        write_table_from_pandas(pd.DataFrame({chave: demais_parte.DE_EXAME.str.contains(chave).to_numpy() for chave in exames}),
                                "../data/interim/fleury_exames_analito/exames/onehot_encoding/bool_parte"+str(i)+".parquet")